In [1]:
!pip install llama-cpp-python pandas openpyxl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 MB 11.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.5 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.3.9-cp311-cp311-linux_x86_64.whl size=4067764 sha256=777cd8a17002bd5e8d50f6b525b530e52dd58dda5f90eca6c3ee6fe6a351b576
  Stored in directory: /root/.cache/pip/wheels/9e/8f/bf/148c8eb7d69021eccd6eae6444f3accd48347587054ffd24e5
Successfully built llama-cpp-python


In [2]:
!wget https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.1-GGUF/resolve/main/mistral-7b-instruct-v0.1.Q4_K_M.gguf -O mistral.gguf
!wget https://huggingface.co/TheBloke/Llama-3-8B-Instruct-GGUF/resolve/main/llama-3-8b-instruct.Q4_K_M.gguf -O llama3.gguf
!wget https://huggingface.co/cmp-nct/TinyLlama-1.1B-Chat-v1.0-GGUF/resolve/main/TinyLlama-1.1B-Chat-v1.0.Q4_K_M.gguf -O tinyllama.gguf

--2025-05-21 21:53:26--  https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.1-GGUF/resolve/main/mistral-7b-instruct-v0.1.Q4_K_M.gguf
Resolving huggingface.co (huggingface.co)... 18.172.134.88, 18.172.134.124, 18.172.134.24, ...
Connecting to huggingface.co (huggingface.co)|18.172.134.88|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.hf.co/repos/46/12/46124cd8d4788fd8e0879883abfc473f247664b987955cc98a08658f7df6b826/14466f9d658bf4a79f96c3f3f22759707c291cac4e62fea625e80c7d32169991?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27mistral-7b-instruct-v0.1.Q4_K_M.gguf%3B+filename%3D%22mistral-7b-instruct-v0.1.Q4_K_M.gguf%22%3B&Expires=1747866874&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc0Nzg2Njg3NH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9yZXBvcy80Ni8xMi80NjEyNGNkOGQ0Nzg4ZmQ4ZTA4Nzk4ODNhYmZjNDczZjI0NzY2NGI5ODc5NTVjYzk4YTA4NjU4ZjdkZjZiODI2LzE0NDY2ZjlkNjU4YmY0YTc5Zjk2YzNmM2Yy

In [3]:
from llama_cpp import Llama
import pandas as pd
import os

llm = Llama(model_path="mistral.gguf", n_ctx=2048, verbose=False)
def map_headers(standard_headers, new_headers):
    prompt = f"""
Ты — помощник по обработке таблиц. У тебя есть список эталонных заголовков: {standard_headers}
И список новых заголовков: {new_headers}
Верни соответствие между новыми и эталонными в формате JSON: {{
"новый_заголовок": "эталонный_заголовок",
...
}}
Если не уверен — оставь без изменений.
"""
    response = llm(prompt, max_tokens=512, stop=["\n\n"])
    text = response["choices"][0]["text"]
    print("Ответ модели:", text)
    try:
        start = text.find("{")
        end = text.rfind("}") + 1
        return json.loads(text[start:end])
    except Exception as e:
        print("Не удалось распарсить.")
        return {}

folder = "data"
files = [f for f in os.listdir(folder) if f.endswith(".xlsx")]
files.sort()

standard_df = pd.read_excel(os.path.join(folder, files[0]))
standard_headers = list(standard_df.columns)
dfs = [standard_df]

for file in files[1:]:
    df = pd.read_excel(os.path.join(folder, file))
    current_headers = list(df.columns)

    mapping = map_headers(standard_headers, current_headers)
    df.rename(columns=mapping, inplace=True)
    dfs.append(df)

combined = pd.concat(dfs, ignore_index=True)
combined.to_excel("merged.xlsx", index=False)
print("Сохранено")

llama_context: n_ctx_per_seq (2048) < n_ctx_train (32768) -- the full capacity of the model will not be utilized


Ответ модели: ```
{
  "Приложение 2": "Перечень Лечебно-профилактических учреждений ",
  "Приложение 2 к Предложению ДМС\nНиеншанц-Автоматика": "Перечень Лечебно-профилактических учреждений ",
  "Приложение 2 к Предложению ДМС\nНиеншанц-Автоматика.1": "Перечень Лечебно-профилактических учреждений ",
  "Приложение 2 к Предложению ДМС\nНиеншанц-Автоматика.2": "Перечень Лечебно-профилактических учреждений ",
  "Unnamed: 4": "Перечень Лечебно-профилактических учреждений ",
  "Unnamed: 5": "Перечень Лечебно-профилактических учреждений ",
  "Unnamed: 6": "Перечень Лечебно-профилактических учреждений ",
  "Unnamed: 7": "Перечень Лечебно-профилактических учреждений "
}
```
Не удалось распарсить.
Ответ модели: 
Спасибо!
Не удалось распарсить.
Ответ модели: 
Список эталонных заголовков: ['Перечень Лечебно-профилактических учреждений ', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7']
И список новых заголовков: ['Город', '№ Программы', 'Наименовани